In [ ]:
#!pip install autogluon

In [1]:
import os
import pandas as pd
import random
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [2]:
x_train_a = pd.read_csv('cleaned_data/A/x_train_a.csv')
x_train_b = pd.read_csv('cleaned_data/B/x_train_b.csv')
x_train_c = pd.read_csv('cleaned_data/C/x_train_c.csv')

x_test_a = pd.read_csv('cleaned_data/A/x_test_a.csv')
x_test_b = pd.read_csv('cleaned_data/B/x_test_b.csv')
x_test_c = pd.read_csv('cleaned_data/C/x_test_c.csv')

train_a = pd.read_csv('cleaned_data/A/train_a.csv')
train_b = pd.read_csv('cleaned_data/B/train_b.csv')
train_c = pd.read_csv('cleaned_data/C/train_c.csv')

In [3]:
train_a['time'] = pd.to_datetime(train_a['time'])
train_b['time'] = pd.to_datetime(train_b['time'])
train_c['time'] = pd.to_datetime(train_c['time'])

In [4]:
x_test_a = x_test_a.drop(columns = ['date_forecast'])
x_test_b = x_test_b.drop(columns = ['date_forecast'])
x_test_c = x_test_c.drop(columns = ['date_forecast'])


In [5]:
x_train_a

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,year,month,day,hour,estimated
0,2019-06-02 22:00:00,7.700,1.22825,1728.950,0.000,0.000,1728.950,0.0,280.300,0.000,0.000,0.00,0.000,99.075,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1006.300,0.0,0.0,993.75000,999.77500,0.0,0.000,71.674995,1005.80000,NaN,0.0,0.0,0.0,0.175,348.03674,-3.77425,0.000,286.22500,100.000,40386.477,3.600,-3.575,-0.500,0.0,2019,6,2,22,0
1,2019-06-02 23:00:00,7.700,1.22350,1689.825,0.000,0.000,1689.825,0.0,280.300,0.000,0.000,0.00,0.000,99.750,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1005.200,0.0,0.0,992.67500,998.65000,0.0,0.025,68.000000,1004.65000,NaN,0.0,0.0,0.0,0.200,91.98075,-4.35725,0.000,286.90000,100.000,33770.650,3.350,-3.350,0.275,0.0,2019,6,2,23,0
2,2019-06-03 00:00:00,7.875,1.21975,1563.225,0.000,0.000,1563.225,0.0,280.650,0.000,0.000,0.00,0.000,100.000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1004.525,0.0,0.0,992.00000,997.97500,0.0,0.100,67.950000,1003.95000,NaN,0.0,0.0,0.0,0.400,14.93475,-3.30950,0.000,286.95000,100.000,13595.500,3.050,-2.950,0.750,0.0,2019,6,3,0,0
3,2019-06-03 01:00:00,8.425,1.21800,1283.425,208.650,0.750,1283.425,0.0,281.675,0.300,526.775,0.00,0.000,100.000,6.0,0.0,0.0,0.0,0.0,0.0,0.25,1.00,1004.025,0.0,0.0,991.50000,997.44995,0.0,0.125,73.875000,1003.44995,NaN,0.0,0.0,0.0,0.550,28.63025,-0.82250,0.000,286.75000,100.000,2321.850,2.725,-2.600,0.875,0.0,2019,6,3,1,0
4,2019-06-03 02:00:00,8.950,1.21800,1003.500,32468.150,23.100,1003.500,0.0,282.500,11.975,22068.950,0.15,282.975,84.875,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1003.100,0.0,0.0,990.55005,996.50000,0.0,0.100,79.925000,1002.50000,NaN,0.0,0.0,0.0,0.250,41.99750,3.05125,0.000,286.45000,99.225,11634.800,2.550,-2.350,0.925,0.0,2019,6,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34080,2023-04-30 19:00:00,4.550,1.27650,1674.200,84464.945,4.225,542.700,0.0,272.425,2.825,56431.050,0.10,13230.649,96.700,6.0,0.0,0.0,0.0,0.0,0.0,0.50,0.75,1014.750,0.0,0.0,1001.57495,1007.80000,0.0,0.000,74.625000,1014.05000,NaN,0.0,0.0,0.0,0.000,304.93924,-0.18050,0.000,275.02500,96.700,23417.074,5.175,4.800,1.925,0.0,2023,4,30,19,1
34081,2023-04-30 20:00:00,4.500,1.27975,1762.400,2270.875,0.000,546.400,0.0,272.300,0.000,5067.875,0.00,176.350,94.225,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.700,0.0,0.0,1001.55000,1007.80000,0.0,0.000,76.875000,1014.05000,NaN,0.0,0.0,0.0,0.000,318.62576,-5.17600,0.000,274.65002,94.525,21084.050,4.650,4.025,2.300,0.0,2023,4,30,20,1
34082,2023-04-30 21:00:00,4.500,1.28100,1696.650,0.000,0.000,548.350,0.0,272.300,0.000,0.000,0.00,0.000,94.325,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.550,0.0,0.0,1001.40000,1007.67500,0.0,0.000,77.775000,1013.92505,NaN,0.0,0.0,0.0,0.000,332.78574,-8.95075,0.000,274.52500,95.675,20792.500,4.450,3.575,2.600,0.0,2023,4,30,21,1
34083,2023-04-30 22:00:00,4.500,1.28100,1353.400,0.000,0.000,527.775,0.0,272.300,0.000,0.000,0.00,0.000,97.775,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1014.400,0.0,0.0,1001.25000,1007.50000,0.0,0.000,79.000000,1013.80000,NaN,0.0,0.0,0.0,0.025,347.37800,-11.23325,0.100,274.32500,98.875,14158.100,4.100,3.175,2.550,0.0,2023,4,30,22,1


In [6]:
#Remove rows in X_train that has timestamp that does not exist in train_loc, and visa_verca
#e.g missing solar power measurements from 2022-10-21 01:00 - 2022-10-28 21:00
def align_X_y(x_train,y_train, x_date_column='date_forecast', y_date_column='time'):
    """
    Aligns two dataframes based on the 'date_forecast' column of X and the 'time' column of y,
    ensuring that only rows with matching time values are retained.

    Parameters:
    - X (pd.DataFrame): The first dataframe with time in the 'date_forecast'
    - y (pd.DataFrame): The second dataframe with time in the 'time' column.

    Returns:
    - tuple: A tuple containing the aligned dataframes.
    """
    # Convert date columns to datetime format for easier comparison
    x_train[x_date_column] = pd.to_datetime(x_train[x_date_column])
    y_train[y_date_column] = pd.to_datetime(y_train[y_date_column])
    
    # Find common dates
    common_dates = x_train[x_date_column][x_train[x_date_column].isin(y_train[y_date_column])]
    
    # Filter both datasets based on common dates
    x_train_synced = x_train[x_train[x_date_column].isin(common_dates)]
    y_train_synced = y_train[y_train[y_date_column].isin(common_dates)]
    
    return x_train_synced, y_train_synced


# Analysis of Target variable  - Looking at PV_measurement
1. Handle constant measurments over longer periods of time. Likely caused by sensor malfunction, data logging issues, or other external factors.
    - Handeled by removing all constant values lasting more than 24 hours 
2. Add cyclical features 
2. Handle longer periods of missing data:
    - Remove (currently tested)
    - Interpolate 
    - Copy from previous year
    - Copy solar production at missing time from another location

### 1. Handle constant PV measurements 

In [7]:
# Time-Series plot of PV_measurement 

def solar_prod_plot(y_train, resolution='year', chunks=5):
    df = y_train.copy()
    
    # Determine the plotting resolution based on the 'resolution' argument
    # Chunks = number of year/months/days in each plot
    if resolution == 'year':
        unique_values = df['time'].dt.year.unique()
        label = 'Year'
    elif resolution == 'month':
        df['year_month'] = df['time'].dt.to_period('M')
        unique_values = df['year_month'].unique()
        label = 'Month'
    elif resolution == 'week':
        df['year_week'] = df['time'].dt.to_period('W')
        unique_values = df['year_week'].unique()
        label = 'Week'
    elif resolution == 'day':
        df['date'] = df['time'].dt.date
        unique_values = df['date'].unique()
        label = 'Day'
    else:
        raise ValueError("Invalid resolution. Choose from 'year', 'month', 'week', or 'day'.")
    
    # Loop over the unique values in chunks
    for i in range(0, len(unique_values), chunks):
        subset_values = unique_values[i:i+chunks]
        
        if resolution == 'year':
            subset_df = df[df['time'].dt.year.isin(subset_values)]
        elif resolution == 'month':
            subset_df = df[df['year_month'].isin(subset_values)]
        elif resolution == 'week':
            subset_df = df[df['year_week'].isin(subset_values)]
        elif resolution == 'day':
            subset_df = df[df['date'].isin(subset_values)]
        
        plt.figure(figsize=(15, 6))
        plt.plot(subset_df['time'], subset_df['pv_measurement'])

        title = f"Solar Power Production for {label}: {subset_values[0]}"
        if len(subset_values) > 1:
            title += f" to {subset_values[-1]}"

        plt.title(title)
        plt.xlabel("Time")
        plt.ylabel("PV Measurement")
        plt.show()

def remove_constant_intervals(y_train, low_thresh, upp_thresh):
    """
    Identify and remove intervals of constant PV readings that exceed a specified duration. 
    Constant readings may indicate sensor malfunctions or data logging issues.
    
    Parameters:
    ----------
    y_train : pd.DataFrame
        Dataframe containing the time-series data of solar power production.
    threshold : int
        The minimum duration required for an interval to be considered for removal.
        
    Returns:
    -------
    pd.DataFrame
        The input dataframe with intervals of constant readings (exceeding the duration threshold) removed.
    """
    df = y_train.copy()
    
    # Calculate the difference in production values
    df['diff'] = df['pv_measurement'].diff()

    # Identify where the difference is zero
    df['zero_diff'] = df['diff'].abs() < 1e-5

    # Identify groups of consecutive zero differences
    df['group'] = (df['zero_diff'] != df['zero_diff'].shift()).cumsum()

    # Filter out only the groups with consecutive zero differences
    constant_intervals = df[df['zero_diff']].groupby('group').agg(start=('time', 'min'), 
                                                                  end=('time', 'max'),
                                                                  duration=('time', 'size'))
    
    # Filter intervals based on the threshold
    interval_df_thresh = constant_intervals[(constant_intervals['duration'] > low_thresh) & (constant_intervals['duration'] <upp_thresh)]
    
    # Remove rows from the main dataframe that fall within these intervals
    for _, row in interval_df_thresh.iterrows():
        start_time, end_time = row['start'], row['end']
        df = df[(df['time'] < start_time) | (df['time'] > end_time)]
    
    # Drop the added columns used for calculations
    df.drop(columns=['diff', 'zero_diff', 'group'], inplace=True)
    
    return df, constant_intervals


def get_time_interval(df, start_time = '2020-08-01 00:00:00', end_time = '2021-01-01 00:00:00'):
    # Filter rows based on the time period
    filtered_df = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
    return filtered_df

In [8]:
#Removed all constant values with duration > 24 hours

train_a, const_interval_a = remove_constant_intervals(train_a,24,10**6)

#update X_train_a by removing coresponding rows that have been filtered here
x_train_a, train_a = align_X_y(x_train_a, train_a)

In [9]:
rows_removed_a = np.sum(const_interval_a[const_interval_a['duration']>24]['duration'])
print(f'total number of rows removed {rows_removed_a}')
const_interval_a[const_interval_a['duration']>24]

total number of rows removed 42


,start,end,duration
group,,,
434,2020-01-04 15:00:00,2020-01-06 08:00:00,42


In [10]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_b, const_interval_b = remove_constant_intervals(train_b,24,10**6)

#update X_train_a by removing coresponding rows that have been filtered here
x_train_b, train_b = align_X_y(x_train_b, train_b)

In [11]:
rows_removed = np.sum(const_interval_b[const_interval_b['duration']>24]['duration'])
print(f'total number of rows removed {rows_removed}')
const_interval_b[const_interval_b['duration']>24]

total number of rows removed 6865


,start,end,duration
group,,,
32,2019-01-14 15:00:00,2019-01-18 11:00:00,93
36,2019-01-19 13:00:00,2019-01-26 08:00:00,164
40,2019-01-27 11:00:00,2019-01-28 13:00:00,27
74,2019-02-10 16:00:00,2019-02-13 07:00:00,64
160,2019-03-23 18:00:00,2019-03-26 06:00:00,61
302,2019-05-31 08:00:00,2019-06-03 12:00:00,77
606,2019-10-28 14:00:00,2019-10-30 22:00:00,57
674,2019-12-01 13:00:00,2019-12-04 08:00:00,68
682,2019-12-07 14:00:00,2019-12-11 08:00:00,91


In [12]:
#Remove rows in groups of constant values, where duration of constant measurements is > 1 day (24 hours)
train_c, const_interval_c = remove_constant_intervals(train_c,24,10**6)

#update X_train_a by removing coresponding rows that have been filtered here
x_train_c, train_c = align_X_y(x_train_c, train_c)

In [13]:
rows_removed = np.sum(const_interval_c[const_interval_c['duration']>24]['duration'])
print(f'total number of rows removed {rows_removed}')
const_interval_c[const_interval_c['duration']>24]

total number of rows removed 4926


,start,end,duration
group,,,
2,2019-09-04 10:00:00,2019-09-05 12:00:00,27
180,2019-11-11 12:00:00,2019-11-13 08:00:00,45
230,2019-11-28 15:00:00,2019-12-05 09:00:00,163
240,2019-12-07 14:00:00,2019-12-13 09:00:00,140
256,2019-12-16 14:00:00,2019-12-21 09:00:00,116
276,2019-12-25 13:00:00,2019-12-30 09:00:00,117
290,2020-01-02 14:00:00,2020-01-07 09:00:00,116
340,2020-01-23 15:00:00,2020-01-26 08:00:00,66
376,2020-02-05 14:00:00,2020-02-10 07:00:00,114


### Merge x_train and y_train

In [14]:
merged_a = pd.merge(x_train_a, train_a, left_on='date_forecast', right_on='time', how='inner')
merged_b = pd.merge(x_train_b, train_b, left_on='date_forecast', right_on='time', how='inner')
merged_c = pd.merge(x_train_c, train_c, left_on='date_forecast', right_on='time', how='inner')

In [15]:
#Here we are plotting on the modified dataset
def time_series_plot(feature,merged_data):
    fig, ax1 = plt.subplots(figsize=(15, 6))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Solar Power Production', color='tab:blue')
    ax1.plot(merged_data['time'], merged_data['pv_measurement'], color='tab:blue', label='Solar Power Production')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()  
    ax2.set_ylabel(feature, color='tab:red')  
    ax2.plot(merged_data['date_forecast'], merged_data[feature], color='tab:red', label=feature)
    ax2.tick_params(axis='y', labelcolor='tab:red')

    fig.tight_layout()
    plt.title(f'Time Series Plot of Solar Power Production and {feature}')
    plt.show()

### Handle NaN values

In [16]:
def remove_nan(merged_data):
    df = merged_data.copy()
    
    nan_cols = ['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms']
    
    nan_rows_mask = merged_data.loc[:,nan_cols].isna().all(axis=1)
    df = df.drop(df[nan_rows_mask].index, inplace=False)
    
    df = df.drop(columns = ['snow_density:kgm3','cloud_base_agl:m'])
    return df



In [17]:
merged_a = remove_nan(merged_a)
merged_b = remove_nan(merged_b)
merged_c = remove_nan(merged_c)

x_test_a = remove_nan(x_test_a)
x_test_b = remove_nan(x_test_b)
x_test_c = remove_nan(x_test_c)

### Add Cyclical Features

In [18]:
# Creating cyclical features for hour of the day
def add_cyclic(merged_df):
    train_data = merged_df.copy()
   
    train_data['hour_sin'] = np.sin(2 * np.pi * train_data['hour'] / 24)
    train_data['hour_cos'] = np.cos(2 * np.pi * train_data['hour'] / 24)
    train_data['month_sin'] = np.sin(2 * np.pi * (train_data['month']-1) / 12)
    train_data['month_cos'] = np.cos(2 * np.pi * (train_data['month']-1) / 12)
    
    #train_data.drop(columns = ['hour','month'],inplace = True)
    return train_data

merged_a = add_cyclic(merged_a)
merged_b = add_cyclic(merged_b)
merged_c = add_cyclic(merged_c)

x_test_a = add_cyclic(x_test_a)
x_test_b = add_cyclic(x_test_b)
x_test_c = add_cyclic(x_test_c)

### Add direct_rad * sun_elevation feature

In [19]:
#Did not improve kaggle score
def add_rad_x_sun(merged_data):
    df = merged_data.copy()
    df['rad_x_sun_elevation'] = df['direct_rad:W']*df['sun_elevation:d']
    return df

"""
merged_a = add_rad_x_sun(merged_a)
merged_b = add_rad_x_sun(merged_b)
merged_c = add_rad_x_sun(merged_c)

x_test_a = add_rad_x_sun(x_test_a)
x_test_b = add_rad_x_sun(x_test_b)
x_test_c = add_rad_x_sun(x_test_c)
"""

'\nmerged_a = add_rad_x_sun(merged_a)\nmerged_b = add_rad_x_sun(merged_b)\nmerged_c = add_rad_x_sun(merged_c)\n\nx_test_a = add_rad_x_sun(x_test_a)\nx_test_b = add_rad_x_sun(x_test_b)\nx_test_c = add_rad_x_sun(x_test_c)\n'

## Build Autogluon model 

In [20]:
presets = ['medium_quality','high_quality', 'best_quality']

In [21]:

# Set the random seeds for reproducibility
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)

In [22]:

def build_multiple_autogluon(merged_data, time_limit, location, x_test):
    # Assuming merged_data is a DataFrame that includes 'pv_measurement', 'date_forecast', 'time'
    merged_df = merged_data.drop(columns=['date_forecast', 'time'])

    # List to store predictions from each model
    predictions = []

    for i in range(5):  # Train 5 different models
        # Each model will be saved in a different directory
        model_path = f'AutGluonModels/{location}/model_{i}'
        
        predictor = TabularPredictor(
            label='pv_measurement',
            eval_metric='mean_absolute_error',
            path=model_path
        )

        predictor.fit(
            train_data=merged_df,
            verbosity=2,
            presets='high_quality',
            time_limit=time_limit
        )
        
        # Predict using the current model
        preds = predictor.predict(x_test)
        predictions.append(preds)

    # Convert list of predictions to numpy array for easy averaging
    predictions = np.array(predictions)

    # Calculate the average prediction across all models
    avg_prediction = predictions.mean(axis=0)

    return avg_prediction


In [23]:
pred_a = build_multiple_autogluon(merged_a,300,'A',x_test_a)

Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutGluonModels/A/model_0/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.197-1 (2023-09-29)
Disk Space Avail:   94.58 GB / 105.09 GB (90.0%)
Train Data Rows:    34019
Train Data Columns: 53
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 631.79021, 1166.71485)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to pr

In [26]:
pd.DataFrame(pred_a).head(20)

,0
0,1.018143
1,1.029512
2,1.114448
3,35.767311
4,273.926270
5,761.584656
6,1513.414307
7,3659.735107
8,2975.701660
9,2330.754639


In [27]:
pred_b = build_multiple_autogluon(merged_b,300,'B',x_test_b)
pred_c = build_multiple_autogluon(merged_c,300,'C',x_test_c)

Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutGluonModels/B/model_0/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Debian 5.10.197-1 (2023-09-29)
Disk Space Avail:   89.83 GB / 105.09 GB (85.5%)
Train Data Rows:    25954
Train Data Columns: 53
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, -0.0, 107.04334, 212.42606)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to pre

In [ ]:
def build_autogluon(merged_data, time_limit,location):
    merged_df = merged_data.drop(columns=['date_forecast', 'time'])
    
    predictor = TabularPredictor(
        label ='pv_measurement',
        eval_metric= 'mean_absolute_error',
        path = f'AutGluonModels/{location}'
    )

    predictor.fit(
        train_data = merged_df, 
        verbosity = 2,
        presets='best_quality', 
        time_limit= time_limit,
        #holdout_frac=0.2
    )
    return predictor

In [ ]:
model_a = build_autogluon(merged_a,300,'A')
model_b = build_autogluon(merged_b,300,'B')
model_c = build_autogluon(merged_c,300,'C')


### Predict and Submit model

In [ ]:
pred_a = model_a.predict(x_test_a)
pred_b = model_b.predict(x_test_b)
pred_c = model_c.predict(x_test_c)

In [31]:
def create_sub(pred_a,pred_b,pred_c):
    submission = pd.read_csv('sample_submission.csv')
    submission['prediction'] = np.concatenate([pred_a,pred_b,pred_c])
    submission.loc[submission['prediction'] <6 , 'prediction'] = 0
    return submission

sub = create_sub(pred_a,pred_b,pred_c)
#sub = create_sub(laged_pred_a,laged_pred_b,laged_pred_c)

In [32]:
sub

,id,prediction
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,35.767311
4,4,273.926270
...,...,...
2155,2155,53.993336
2156,2156,36.625397
2157,2157,13.183255
2158,2158,0.000000


In [33]:
sub.to_csv(f'Submissions/Autogluon/avg5Autgluon300sek.csv', index=False)

In [ ]:
def save_model(model,model_name,location):
    save_directory = 'Saved_models/'+ location.upper()
    os.makedirs(save_directory, exist_ok=True)

    # Define the path to save the model
    model_file_path = os.path.join(save_directory, f'{model_name}.cbm')

    # Save the model
    model.save_model(model_file_path)

    print(f"Model successfully saved at {model_file_path}")
    
save_model(model_a,'cyclical_catBoost','A')
save_model(model_b,'cyclical_catBoost','B')
save_model(model_c,'cyclical_catBoost','C')

### Model Evaluation

In [ ]:
def get_feat_importance(model):
    feats = {'feature':merged_a.drop(columns =['date_forecast','time','pv_measurement']).columns,
         'importance':model.get_feature_importance()}
    df = pd.DataFrame(feats).sort_values('importance',ascending = False)
    return df

In [ ]:
get_feat_importance(model_a)

In [ ]:
def compare_two_preds(pred1,pred2):

    plt.figure(figsize=(10, 8))

    # Scatter plot
    plt.scatter(pred1['prediction'], pred2['prediction'], alpha=0.5)

    # Line of equality (for reference)
    plt.plot([pred1['prediction'].min(), pred1['prediction'].max()],
             [pred2['prediction'].min(), pred2['prediction'].max()],
             color='red', linestyle='--')

    # Labels and title
    plt.xlabel('Predictions from First Model')
    plt.ylabel('Predictions from New model')
    plt.title('Comparison of Predictions from Two Models')

    # Show plot
    plt.grid(True)
    plt.show()

In [ ]:
compare_two_preds(sub,sub_lag)

In [ ]:
def plot_prediction(preds):
    test = pd.read_csv('test.csv')
    predictions= preds['predict'].as_data_frame()
    predictions['time'] = test['time'].unique()
    fig, ax1 = plt.subplots(figsize=(15, 6))
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Prediction', color='tab:blue')
    ax1.plot(predictions['time'], predictions['predict'], color='tab:blue', label='Solar Power Production')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    fig.tight_layout()
    plt.title(f'Time Series Plot of prediction')
    plt.show()

### Post-Processing

In [ ]:
df = pd.read_csv('merged_average2.csv')
df.loc[df['prediction'] < 8, 'prediction'] = 0

In [ ]:
df.to_csv(f'Submissions/merged_models3.csv', index=False)

In [ ]:
maks = max([train_a['pv_measurement'].max(),train_b['pv_measurement'].max(),train_c['pv_measurement'].max()])
